In [19]:
import pandas as pd
from fuzzywuzzy import fuzz
import string
import pickle as pkl
import numpy as np

TRUE_CUTOFF = 3150
THRESHOLD = 85

In [2]:
replace_char = ["*", "&", "%", "/", "\\"]
strip_char = ["'", "-", ".", "!", ":", ";"]
num_char =  ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0"]
red_flag_char = [" AVE ", " ST ", " AV ", " BLVD ", " BLV ", " RD ", " DR "]
common_errors = {
    "0": "O",
    "1": "l",
    "5": "S",
    "8": "B"
}

In [3]:
# remake the dict
map_dict = {}

# use previous dict
# map_dict = pkl.load(open('trueheaders_match_dict.pkl', 'rb'))

In [4]:
def clean_header(h):
    red_flag = False
    for s in replace_char:
        h = h.replace(s, "")
    for s in strip_char:
        h = h.strip(s)
    cnt = 0
    hl = []
    for c in list(h):
        if c in common_errors: c = common_errors[c]
        if c in num_char: cnt += 1
        hl.append(c)
    h = ''.join(hl).upper()
    for rf in red_flag_char:
        if rf in h: red_flag = True
    if cnt > 3 or red_flag: h = ""
    return h
    
def assign_clean(D):
    return [clean_header(h) for h in D.true_headers]

def score(string1, string2):
    # Scores the fuzzy match between the ocr header and the true header
    return fuzz.ratio(string1,string2)

def match(headers, true_headers, map_dict):
    # Matches ocr headers to true headers
    for header in headers:
        if header not in map_dict:
            score_list = [(score(header, true_header), true_header) for true_header in true_headers]
            sorted_score = sorted(score_list, key = lambda tup: tup[0], reverse=True)
            score_tuple = sorted_score[0]
            if score_tuple[0] > 90:
                print("LOG: Matched! " + header + " and " + score_tuple[1] + " with score " + str(score_tuple[0]))
                map_dict[header] = (score_tuple[0], score_tuple[1], "TRUE")
            else:
                print("LOG: Not matched, " + header + " and " + score_tuple[1] + " with score " + str(score_tuple[0]))
                map_dict[header] = (score_tuple[0], "no_header", "FALSE")
    return map_dict

In [5]:
raw = pd.read_csv("RankedHeaders.csv")[["count", "true_headers"]].dropna().assign(true_headers=assign_clean)

In [6]:
df = raw.drop_duplicates("true_headers")
df = df[df.true_headers.map(lambda h: (len(h) < 150) and (len(h) > 2) and (h is not ""))].reset_index(drop=True).sort_values("count", ascending=False)
true_headers = list(df[:TRUE_CUTOFF].true_headers)
unsure_headers = list(df[TRUE_CUTOFF:].true_headers)

In [7]:
map_dict = match(unsure_headers, true_headers, map_dict)

LOG: Not matched, TOYS  GAMES-WHOLESALE and YAMS-WHOLESALE with score 80
LOG: Matched! HOSPITALS AND DISPENSARII'S  and HOSPITALS AND DISPENSARIES with score 93
LOG: Matched! INLUMBER-RETAIL and LUMBER-RETAIL with score 93
LOG: Matched! PICTURE FRAMES  PICTURES.- DEALERS and PICTURE FRAMES  PICTURES- DEALERS with score 99
LOG: Not matched, CEMENT, CEMENT PAINT  CEMENT PIPE and ENTERTAINMENT BUREAUS with score 52
LOG: Matched! HOTEL AND RESTAURANT SUNNLLES AND EQUIPMENT and HOTEL AND RESTAURANT SUPPLIES AND EQUIPMENT with score 93
LOG: Matched! HOSIERY-WHOLESALE  J OBBERS and HOSIERY-WHOLESALE  JOBBERS with score 98
LOG: Matched! PLORISTS-WHOLESALE and FLORISTS-WHOLESALE with score 94
LOG: Matched! REFRIGERATORS-SALOS AND SERVICE and REFRIGERATORS-SALES AND SERVICE with score 97
LOG: Matched! LAUNDRY SUPPLIES  EQUIPMENT -DEALERS and LAUNDRY SUPPLIES  EQUIPMENT DEALERS with score 99
LOG: Not matched, BROKERS-SGOTTO'N and BROKERS-GOLTTO with score 87
LOG: Matched! AUTOMOBILE ACCESSORIES  

LOG: Matched! BUTTER, CHEESE B; NGGS-RETAIL and BUTTER, CHEESE  NGS-RETAIL with score 95
LOG: Not matched, ICE DEALERS-PEDDLERS SUPPLIED and  JEWELERS, SUPPLIES with score 63
LOG: Not matched, LMOCIBTIIB and SOCIETIES with score 53
LOG: Matched! CONTRACTORS-PAVING  CONTRACTORS-PLASTERLNG and CONTRACTORS-PAVING  CONTRACTORS-PLASTERING with score 98
LOG: Not matched, PAINTERS-AUTO  CARRIAGE and PAINTERS-AUTOMOBILE  CARRIAGE with score 88
LOG: Matched! RADIATOR REPAIRERS- AUTOMOBILE and RADIATOR REPAIRERS-AUTOMOBILE with score 98
LOG: Matched! INSURANCE INVESTIGATIONS and INSURANCE INVESTIGATORS with score 94
LOG: Matched! MONUME NTAL WORK and MONUMENTAL WORK with score 97
LOG: Matched! LLABLLITY INSURANCE and LLABLLITV INSURANCE with score 95
LOG: Not matched, CUTLERY-RETAIL and HOSTERY-RETAIL with score 79
LOG: Not matched, TUBE  TUBING MIRS-METAL and STOVES  RANG IAS-RETAIL with score 61
LOG: Matched! PAINTERS-AUTOMOBLLE  CAR. RIAGE and PAINTERS-AUTOMOBILE  CARRIAGE with score 93
LOG: 

LOG: Matched! SHIN CHANDLERS and SHIP CHANDLERS with score 93
LOG: Matched! CONTRACTORS SUPPLIES AND EQUIPMENT-DEALERS AND REPAIRING and CONTRACTORS SUPPLIES AND EQUIPMENT- DEALERS AND REPAIRING with score 99
LOG: Not matched, ADI NSTERS and ADJ USTERS with score 80
LOG: Not matched, CALCULATING MACHINES and DUPLICATING MACHINES with score 80
LOG: Matched! ROOFING MATERIALS  ASPHALT SHINGLES and ROOTING MATERIALS  ASPHALT SHINGLES with score 97
LOG: Matched! CONTRACTORS -ROAD and CONTRACTORS-ROAD with score 97
LOG: Matched! GRANITE AND MARBLE WORKERS  and GRANITE AND MARBLE WORKERS with score 98
LOG: Matched! DRY GOODS- WHOLESALE and DRY GOODS-WHOLESALE with score 97
LOG: Matched! MUSICAL INSTRUMENT MIRS and MUSICAL INSTRUMENT MFRS with score 96
LOG: Matched! TYPEWFITING MACHINES-DEALERS and TYPEWRITING MACHINES-DEALERS with score 96
LOG: Not matched, TYPE FOUNDERS and IRON FOUNDERS  with score 67
LOG: Not matched, CINDER BLOCKS and CONCRETE BLOCK MFRS with score 63
LOG: Matched! OONTR

LOG: Matched! OIL BURNERS-SALI'B AND SERVICE and OIL BURNERS-SALN AND SERVICE with score 93
LOG: Not matched, CONTRACTORS- ACOUSTICAL and CONTRACTORS- ELECTRICAL with score 83
LOG: Not matched, EMBLEMS and EMBALMERS with score 75
LOG: Matched! PRODUCE DEALERS- WHOL and PRODUCE DEALERS-WHOI with score 93
LOG: Matched! LDTHOGRAPHERS and LITHOGRAPHERS with score 92
LOG: Matched! ARTIFICIAL EVES and ARTIFICIAL EYES with score 93
LOG: Matched! RADIATOR REPAIRERS-AUTOMOHILE and RADIATOR REPAIRERS-AUTOMOBILE with score 97
LOG: Matched! NOVELTIES-W'HOLESALE and NOVELTIES-WHOLESALE with score 97
LOG: Not matched, FOREIGN EXCHANGE AND and POREIGN EXCHANGE with score 83
LOG: Not matched, HEATING CONTRACTORS and EXCAVATING CONTRACTORS with score 88
LOG: Not matched, RAILROAD PASSENGER STATIONS and RADIO BROADCASTING STATIONS with score 74
LOG: Matched! CONTRACTORS and CONTRACTORS  with score 96
LOG: Not matched, ENGMEERS4-CWN and ENGINEERS-CONSULTI with score 65
LOG: Not matched, BOOKLET PRINTERS 

LOG: Not matched, GALCIUM CHLORIDE and VACUUM CLEANERS-DEALERS with score 51
LOG: Matched! PENCIL ' MFRS and PENCIL MFRS with score 92
LOG: Not matched, STAMP DEALERS AND PHILATELIC SUPPLIES and POSTAGE STAMPS AND PHILATELIC SUPPLIES with score 80
LOG: Not matched, ALUMINUM and ALUMINUM   with score 89
LOG: Matched! INVE STME NT COUNSE L and INVESTMENT COUNSEL with score 92
LOG: Not matched, HARDWARE DEALERS-WHOLESALE and HARDWARE DEALERS-WHOLESALE  JOBBERS with score 85
LOG: Not matched, TRANSPORTATLON-PRELUHT-ANTO- MOBILE and TRANSPORTATION-PREIGHT-AUTO- MOBILE with score 89
LOG: Not matched, CABARETS and BARBERS with score 67
LOG: Not matched, PARQUETRY FLOORING  PATENT ATTORNEYS and PATENT ATTORNEYS with score 62
LOG: Not matched, WORSTED GOODS MANUFACTURERS  WORSTED  MERINO GOODS MFRS and WORSTED  MERINO GOODS-MIRS with score 59
LOG: Matched! RECRUITLNG-ARMED SERVICES and RECRUITING-ARMED SERVICES with score 96
LOG: Not matched, UPHOLSTERING MATERIALS  SUP and BUILDING MATERIALS  

...

LOG: Not matched, DRAPERIES AND CURTAIN S and GROCERIES AND MEATS-RETAIL  with score 64
LOG: Not matched, STOCK BROKERS and BONDS  STOCKS-BROKERS with score 71
LOG: Not matched, RAILMADS  and RAILROADS  with score 84
LOG: Not matched, EMPLOYMENT AGENCIES-SECRE- TARIAL PLACEMENT BUREAUS and ENTERTAINMENT BUREAUS with score 56
LOG: Not matched, DERMATOLOGIST and DENTIST with score 60
LOG: Not matched, P-LLLOW SANITIZING and F'PILLOW SANITIZING with score 86
LOG: Not matched, INSURANEE-MARLNE and INSURANCE-MARINE with score 88
LOG: Matched! LOANS -AUTOMOBILE and LOANS-AUTOMOBILE with score 97
LOG: Not matched,  TITLE CO and TITLE INSURANCE with score 58
LOG: Not matched,  PLNS and  LOANS with score 73
LOG: Matched! FLOORS A and FLOORS  with score 93
LOG: Matched! TYPESE'ITING-MACHINE COM- POSITION and TYPESETTING-MACHINE COMPOSITION with score 92
LOG: Not matched, L'PRINTING PLATES and PRINTING PRESS MFRS with score 67
LOG: Not matched, CONCRETE -READY MIXED and ICE CREAM MIX with score 5

LOG: Matched! LEMBALMERS and EMBALMERS with score 95
LOG: Not matched, LCARRIAGE  WAGON MFRS  GALSEMENTS-STEEL and STORAGE-FURS AND GARMENTS with score 50
LOG: Not matched, SHLPPLNG TAG MFRS and SPRING MFRS with score 64
LOG: Not matched, HORSESHOEM  F and HORSESHOERS  HOSIERY-RETAIL with score 55
LOG: Not matched, RETRIGERATORS-ELEC,TRIC  GAS and REFRIGERATORS-ELECTRIC with score 84
LOG: Not matched, BINSO and SIGNS with score 60
LOG: Not matched, TUBE  TUBING MIRS-MEFTAL and STOVES  RANG IAS-RETAIL with score 60
LOG: Matched! TE LEGRAPH COMPANIES and TELEGRAPH COMPANIES with score 97
LOG: Matched! IINSURANCE COMPANIES and INSURANCE COMPANIES with score 97
LOG: Not matched, DOOR MANUFACTURERS- FOLDING and TOOL MANUFACTURERS with score 71
LOG: Not matched, BRLCK PACE  COMMO and BRICK, LIME  CEMENT with score 56
LOG: Not matched, GUTTERS AND CONDUCTOR PIPE -GALVANI ZED STEEL AND COPPER and GUTTERS  CONDUCTOR PIPE  with score 60
LOG: Not matched, CAULKJNG COMPOUNDS AND GUNS and CLEANING 

LOG: Not matched, CLOTHING MFRS-CHILDRE and CLOTHING MFRS-WOMEN'S with score 71
LOG: Not matched, CANDLCS and CHAINS with score 62
LOG: Not matched, CLEANING COMPOUNDS-MANU- FACTURERS and CLEANING COMPOUNDS-MFRS with score 81
LOG: Not matched, WATERPROOFING COMPOSITION and WATERPROOFING with score 68
LOG: Matched! JEWELERS--SPECIAL ORDER WORK and JEWELERS-SPECIAL ORDER WORK with score 98
LOG: Not matched, ANNUITIEB and ANTIQUES with score 59
LOG: Not matched, FIRE PROTECTION SPRINKLER SYSTEMS and FIRE PROTECTION SERVICE with score 71
LOG: Not matched, TBOILER INSTALLATIONS O'CONNOR JOHN F PLUMBING  HEATIM and BOILER INSTALLATIONS AND REPAIRS with score 60
LOG: Not matched, OLASS PINS  RINGS and ASSOC I AT IONS with score 63
LOG: Matched! MOVERS-HOUSIEHOLD GOODS and MOVERS-HOUSEHOLD GOODS with score 98
LOG: Not matched, WOMEN'S SPECIALTY SHOPS and WIRE SPECIALTIES with score 62
LOG: Not matched, L' PRINTERS-OFFSET and  PHOTO-OFFSET with score 65
LOG: Not matched, NET AND SEINE MAKERS an

LOG: Not matched, SPRAY PAINTING and PAINTING with score 73
LOG: Matched! ELECTRLC MOTOR REPAIRS and ELECTRIC MOTOR REPAIRS with score 95
LOG: Not matched,  GLOCKS-D EALECRS-WHOLESALE and GOAL DEALERS-WHOLESALE with score 78
LOG: Not matched, HEATINQ  PLUMBING and PLUMBING with score 64
LOG: Not matched, WULLEYS and BOWLING ALLEYS with score 57
LOG: Not matched, CLOTHING-BOYS'-RETALL and CLOTHING-RETAIL with score 78
LOG: Not matched, PAR TS-MANUFAC TURERS and PARTS MANUFACTURERS with score 90
LOG: Not matched, CLOTHES DRYERS-ELECTRIC and CONTRACTORS-ELECTRICAL with score 67
LOG: Not matched, SAIE DEALERS AND EXPERTS and TIRE DEALERS AND REPAIRERS with score 80
LOG: Not matched, BUJLDING CONTRACTORS and PAINTING CONTRACTORS with score 75
LOG: Not matched, CEMENT COLORS and INVESTMENT COUNSELORS with score 71
LOG: Not matched, WOOLEN GOODS- MANUFACTURERS and WIRE GOODS MANUFACTURERS with score 86
LOG: Not matched, WATER W ORKS SYSTEMS and TEMPERATURE CONTROL SYSTEMS with score 60
LOG: M

LOG: Not matched, STAMNINGS-METAL and STATIONERS-RETAIL with score 69
LOG: Not matched, AUTOMATIC TYPEWRITING and AUTOMOBILE PARKING with score 62
LOG: Not matched, GESIDE-NTIAL PAINTERS and SIGN PAINTERS MRS with score 63
LOG: Not matched, ELECTRIC MOTORS AND  GENERATORS-MFRS and ELECTRIC MOTORS  GENERATORS with score 86
LOG: Not matched, E LASTIC STOCKINGS and STOCKINGS with score 67
LOG: Not matched,  BUS AND TRUCK BATTERIES- HEAVY DUTY and C URTAINS AND DRAPERIES with score 51
LOG: Not matched, LINENS-RETAIL  LINOTYPE  MONOTYPE GOM- POSITIMI and INSTALLMENT GOODS with score 47
LOG: Not matched, LWZ'RUTTERS AND CONDUCTOR PIPE-GALVANIZED STEEL, COPPER AND ALUMINUM and GUTTERS  CONDUCTOR PIPE  with score 50
LOG: Not matched, POSTAUE STAMP COLLECTORS  and PATENT SOLICITORS with score 57
LOG: Not matched, COMPUTER SCHOOLS and GOLF SCHOOLS with score 64
LOG: Matched! CLCTHING-WOMENS-RETAIL and CLOTHING-WOMEN'S-RETAIL with score 93
LOG: Not matched, FENCE ERECTORS GENERAL FENCE SUPPLY COM

In [8]:
pkl.dump(map_dict, open('trueheaders_match_dict.pkl', 'wb'))

In [9]:
def assign_matched(D):
    matched = []
    for h in D.true_headers:
        if h in map_dict: 
            matched.append(map_dict[h][1])
        else: 
            print("Known: ", h, " not in map_dict")
            matched.append(h)
    return matched
def assign_score(D):
    scores = []
    for h in D.true_headers:
        if h in map_dict: 
            scores.append(map_dict[h][0])
        else:
            print("Known: ", h, " not in map_dict")
            scores.append(np.nan)
    return scores
def assign_bool(D):
    is_matched = []
    for h in D.true_headers:
        if h in map_dict:
            is_matched.append(map_dict[h][2])
        else:
            print("Known: ", h, " not in map_dict")
            is_matched.append("KNOWN")
    return is_matched
df = df.assign(matched = assign_matched, score = assign_score, is_matched = assign_bool).sort_values("count", ascending=False)

('Known: ', 'LODGING HOUSES', ' not in map_dict')
('Known: ', 'VARIETY STORES', ' not in map_dict')
('Known: ', 'BARBERS', ' not in map_dict')
('Known: ', 'RESTAURANTS  LUNCH ROOMS', ' not in map_dict')
('Known: ', 'LIQUORS-RETAIL', ' not in map_dict')
('Known: ', 'PHYSICIANS  SURGEONS', ' not in map_dict')
('Known: ', 'SHOE REPAIRERS', ' not in map_dict')
('Known: ', 'LAW OFFICES', ' not in map_dict')
('Known: ', 'LAW BOOKS, LAWYERS', ' not in map_dict')
('Known: ', 'NURSES', ' not in map_dict')
('Known: ', 'MUSIC TEACHERS', ' not in map_dict')
('Known: ', 'DENTISTS', ' not in map_dict')
('Known: ', 'CLERGYME', ' not in map_dict')
('Known: ', 'RESTAURANTS', ' not in map_dict')
('Known: ', 'BEAUTY SHOPS', ' not in map_dict')
('Known: ', 'BARBER SHOPS', ' not in map_dict')
('Known: ', 'CLERGYMEN', ' not in map_dict')
('Known: ', 'LAWYERS', ' not in map_dict')
('Known: ', 'CLUBS', ' not in map_dict')
('Known: ', 'INSURANCE AGENTS', ' not in map_dict')
('Known: ', 'APARTMENT BUILDINGS', '

('Known: ', 'WATERPROOFING', ' not in map_dict')
('Known: ', 'BOX MIRS-PANER', ' not in map_dict')
('Known: ', 'REAL ESTATE AGENTS', ' not in map_dict')
('Known: ', "YARNS'-RETAIL", ' not in map_dict')
('Known: ', 'WASTE DEALERS-COTTON. SILK AND WOOL', ' not in map_dict')
('Known: ', 'KINDER INC', ' not in map_dict')
('Known: ', 'DEPARTMENT STATES', ' not in map_dict')
('Known: ', 'PREFABRICATED BUILDINGS', ' not in map_dict')
('Known: ', 'SMITH', ' not in map_dict')
('Known: ', "F'ATTLOVMEY-AT-LAW", ' not in map_dict')
('Known: ', 'INSURANCE COMPANIES-ANTO- MOBILE', ' not in map_dict')
('Known: ', 'PLUMBING', ' not in map_dict')
('Known: ', 'WASTE DEALERS-COTTON, SILK IND WOOL', ' not in map_dict')
('Known: ', 'ELECTRICAL EQUIPMENT AND SUPPLIES-MIRS', ' not in map_dict')
('Known: ', 'RUG CLEANING  RENOVATING', ' not in map_dict')
('Known: ', 'GENERAL MERCHANDISE-WHOL', ' not in map_dict')
('Known: ', 'S-NNMETAL DEALERS', ' not in map_dict')
('Known: ', 'WINDOW SHADE DEALERS', ' not in

('Known: ', 'TRUCKS- LEASING', ' not in map_dict')
('Known: ', 'TYPESETTING-MACHINE COMPOSITION', ' not in map_dict')
('Known: ', 'SHIPSMITHS  SHIRT MFRS', ' not in map_dict')
('Known: ', 'DISINFECTANT MFRS', ' not in map_dict')
('Known: ', "HARDWARE-RE'I'AIL", ' not in map_dict')
('Known: ', 'INSULATION-ROCK WOOL', ' not in map_dict')
('Known: ', 'PA PER STOCK', ' not in map_dict')
('Known: ', 'BROKERS-STOCKS AND BONDS ', ' not in map_dict')
('Known: ', 'FIRE INSURAHCE', ' not in map_dict')
('Known: ', 'INSTALLMENT GOODS', ' not in map_dict')
('Known: ', 'LADIES TAILORS ', ' not in map_dict')
('Known: ', 'MODEL MAKERS  MONUMENTAL WORKS', ' not in map_dict')
('Known: ', 'REFRIGERATORS-SALES AND SERVICE', ' not in map_dict')
('Known: ', 'STATIONERS-MTG ', ' not in map_dict')
('Known: ', 'INSURANCE COMPANIES-JDABILITV', ' not in map_dict')
('Known: ', 'FURNITURE LOANS', ' not in map_dict')
('Known: ', 'ICE CREAM DEALERS', ' not in map_dict')
('Known: ', 'POULTRY DEALERSU-RETAIL', ' not i

('Known: ', 'AUTOMOBILE WASHING AND POLISHING', ' not in map_dict')
('Known: ', 'MUSIC DEALERS-SHEET', ' not in map_dict')
('Known: ', 'CHAIN MFRS', ' not in map_dict')
('Known: ', 'CONTRACTORS- BUILDLNG-GENERAL', ' not in map_dict')
('Known: ', 'EMBALMERS', ' not in map_dict')
('Known: ', 'ORGANIZATIONS -PATRIOTIC AND VETERANS', ' not in map_dict')
('Known: ', 'SCHOOIS AND COLLEGES', ' not in map_dict')
('Known: ', 'ORGANIZATIONS-VETERANS', ' not in map_dict')
('Known: ', 'HARDWARE MFRS', ' not in map_dict')
('Known: ', 'INSURANCE ADJUSTORS', ' not in map_dict')
('Known: ', 'ENTERTAINMENT BUREAUS', ' not in map_dict')
('Known: ', 'MUSEUMS', ' not in map_dict')
('Known: ', 'ORGANIZATIONS-MISC', ' not in map_dict')
('Known: ', 'WATCH CLOCK AND JEWELRY REPAIRING', ' not in map_dict')
('Known: ', 'TUTORING SERVICE', ' not in map_dict')
('Known: ', 'BUILDING', ' not in map_dict')
('Known: ', 'DAIRY STORES', ' not in map_dict')
('Known: ', 'BOOKBINDERS', ' not in map_dict')
('Known: ', 'HOU

In [10]:
known = df.loc[df.matched != "no_header"].reset_index(drop=True).drop_duplicates().sort_values("count", ascending=False)
unknown = df.loc[df.matched == "no_header"].reset_index(drop=True).drop_duplicates("true_headers").sort_values("count", ascending=False)

In [11]:
# def internal_match(headers, true_headers, map_dict):
#     # Matches ocr headers to true headers
#     for header in headers:
#         true_headers = [h for h in true_headers if h != header]
#         if header not in map_dict:
#             score_list = [(score(header, true_header), true_header) for true_header in true_headers]
#             sorted_score = sorted(score_list, key = lambda tup: tup[0], reverse=True)
#             score_tuple = sorted_score[0]
#             if score_tuple[0] > 90:
#                 print("LOG: Matched! " + header + " and " + score_tuple[1] + " with score " + str(score_tuple[0]))
#                 map_dict[header] = score_tuple
#             else:
#                 print("LOG: Not matched, " + header + " and " + score_tuple[1] + " with score " + str(score_tuple[0]))
#                 map_dict[header] = (score_tuple[0], "no_header")
#     return map_dict
# def assign_matched_internal(D):
#     matched = []
#     for h in D.true_headers:
#         if h in internal_dict: 
#             matched.append(internal_dict[h][1])
#         else: 
#             matched.append(h)
#     return matched
# def assign_score_internal(D):
#     scores = []
#     for h in D.true_headers:
#         if h in internal_dict: 
#             scores.append(internal_dict[h][0])
#         else:
#             scores.append(np.nan)
#     return scores

# internal_dict = {}
# internal_dict = internal_match(list(unknown.true_headers), list(unknown.true_headers), internal_dict)

# internal_matched = unknown
# internal_matched = internal_matched.assign(matched = assign_matched_internal, score = assign_score_internal)
# unknown2 = internal_matched.loc[internal_matched.matched == "no_header"].reset_index(drop=True).drop_duplicates("true_headers")

# internal_dict = internal_match(list(unknown2.true_headers), list(unknown2.true_headers), internal_dict)

# internal_matched2 = unknown2
# internal_matched2 = internal_matched2.assign(matched = assign_matched_internal, score = assign_score_internal)
# unknown3 = internal_matched2.loc[internal_matched2.matched == "no_header"].reset_index(drop=True).drop_duplicates("true_headers")

# unknown3

In [12]:
def calculate_scores(df):
    l = len(df.true_headers)
    score_matrix = np.zeros((l,l))
    i = 0
    for h in df.true_headers:
        score_matrix[i, :] = df.true_headers.apply(score, args=(h,)).values
        print("Row number: {} of {}".format(i, l - 1))
        i += 1
    return score_matrix

def remove_repeat(df, scores):
    prelist = list(df.true_headers)
    i = 0
    for h in df.true_headers:
        j = 0
        for removal in scores[i, :]:
            if removal and j > i:
                try:
                    df = df[df["true_headers"] != prelist[j]]
                    print("LOG: " + prelist[j] + " deleted!")
                except KeyError:
                    print("LOG: " + prelist[j] + " Already deleted")
            j += 1
        i += 1
    return df

In [13]:
scores = calculate_scores(unknown)

Row number: 0 of 7661
Row number: 1 of 7661
Row number: 2 of 7661
Row number: 3 of 7661
Row number: 4 of 7661
Row number: 5 of 7661
Row number: 6 of 7661
Row number: 7 of 7661
Row number: 8 of 7661
Row number: 9 of 7661
Row number: 10 of 7661
Row number: 11 of 7661
Row number: 12 of 7661
Row number: 13 of 7661
Row number: 14 of 7661
Row number: 15 of 7661
Row number: 16 of 7661
Row number: 17 of 7661
Row number: 18 of 7661
Row number: 19 of 7661
Row number: 20 of 7661
Row number: 21 of 7661
Row number: 22 of 7661
Row number: 23 of 7661
Row number: 24 of 7661
Row number: 25 of 7661
Row number: 26 of 7661
Row number: 27 of 7661
Row number: 28 of 7661
Row number: 29 of 7661
Row number: 30 of 7661
Row number: 31 of 7661
Row number: 32 of 7661
Row number: 33 of 7661
Row number: 34 of 7661
Row number: 35 of 7661
Row number: 36 of 7661
Row number: 37 of 7661
Row number: 38 of 7661
Row number: 39 of 7661
Row number: 40 of 7661
Row number: 41 of 7661
Row number: 42 of 7661
Row number: 43 of 766

Row number: 352 of 7661
Row number: 353 of 7661
Row number: 354 of 7661
Row number: 355 of 7661
Row number: 356 of 7661
Row number: 357 of 7661
Row number: 358 of 7661
Row number: 359 of 7661
Row number: 360 of 7661
Row number: 361 of 7661
Row number: 362 of 7661
Row number: 363 of 7661
Row number: 364 of 7661
Row number: 365 of 7661
Row number: 366 of 7661
Row number: 367 of 7661
Row number: 368 of 7661
Row number: 369 of 7661
Row number: 370 of 7661
Row number: 371 of 7661
Row number: 372 of 7661
Row number: 373 of 7661
Row number: 374 of 7661
Row number: 375 of 7661
Row number: 376 of 7661
Row number: 377 of 7661
Row number: 378 of 7661
Row number: 379 of 7661
Row number: 380 of 7661
Row number: 381 of 7661
Row number: 382 of 7661
Row number: 383 of 7661
Row number: 384 of 7661
Row number: 385 of 7661
Row number: 386 of 7661
Row number: 387 of 7661
Row number: 388 of 7661
Row number: 389 of 7661
Row number: 390 of 7661
Row number: 391 of 7661
Row number: 392 of 7661
Row number: 393 

Row number: 699 of 7661
Row number: 700 of 7661
Row number: 701 of 7661
Row number: 702 of 7661
Row number: 703 of 7661
Row number: 704 of 7661
Row number: 705 of 7661
Row number: 706 of 7661
Row number: 707 of 7661
Row number: 708 of 7661
Row number: 709 of 7661
Row number: 710 of 7661
Row number: 711 of 7661
Row number: 712 of 7661
Row number: 713 of 7661
Row number: 714 of 7661
Row number: 715 of 7661
Row number: 716 of 7661
Row number: 717 of 7661
Row number: 718 of 7661
Row number: 719 of 7661
Row number: 720 of 7661
Row number: 721 of 7661
Row number: 722 of 7661
Row number: 723 of 7661
Row number: 724 of 7661
Row number: 725 of 7661
Row number: 726 of 7661
Row number: 727 of 7661
Row number: 728 of 7661
Row number: 729 of 7661
Row number: 730 of 7661
Row number: 731 of 7661
Row number: 732 of 7661
Row number: 733 of 7661
Row number: 734 of 7661
Row number: 735 of 7661
Row number: 736 of 7661
Row number: 737 of 7661
Row number: 738 of 7661
Row number: 739 of 7661
Row number: 740 

Row number: 1040 of 7661
Row number: 1041 of 7661
Row number: 1042 of 7661
Row number: 1043 of 7661
Row number: 1044 of 7661
Row number: 1045 of 7661
Row number: 1046 of 7661
Row number: 1047 of 7661
Row number: 1048 of 7661
Row number: 1049 of 7661
Row number: 1050 of 7661
Row number: 1051 of 7661
Row number: 1052 of 7661
Row number: 1053 of 7661
Row number: 1054 of 7661
Row number: 1055 of 7661
Row number: 1056 of 7661
Row number: 1057 of 7661
Row number: 1058 of 7661
Row number: 1059 of 7661
Row number: 1060 of 7661
Row number: 1061 of 7661
Row number: 1062 of 7661
Row number: 1063 of 7661
Row number: 1064 of 7661
Row number: 1065 of 7661
Row number: 1066 of 7661
Row number: 1067 of 7661
Row number: 1068 of 7661
Row number: 1069 of 7661
Row number: 1070 of 7661
Row number: 1071 of 7661
Row number: 1072 of 7661
Row number: 1073 of 7661
Row number: 1074 of 7661
Row number: 1075 of 7661
Row number: 1076 of 7661
Row number: 1077 of 7661
Row number: 1078 of 7661
Row number: 1079 of 7661


Row number: 1372 of 7661
Row number: 1373 of 7661
Row number: 1374 of 7661
Row number: 1375 of 7661
Row number: 1376 of 7661
Row number: 1377 of 7661
Row number: 1378 of 7661
Row number: 1379 of 7661
Row number: 1380 of 7661
Row number: 1381 of 7661
Row number: 1382 of 7661
Row number: 1383 of 7661
Row number: 1384 of 7661
Row number: 1385 of 7661
Row number: 1386 of 7661
Row number: 1387 of 7661
Row number: 1388 of 7661
Row number: 1389 of 7661
Row number: 1390 of 7661
Row number: 1391 of 7661
Row number: 1392 of 7661
Row number: 1393 of 7661
Row number: 1394 of 7661
Row number: 1395 of 7661
Row number: 1396 of 7661
Row number: 1397 of 7661
Row number: 1398 of 7661
Row number: 1399 of 7661
Row number: 1400 of 7661
Row number: 1401 of 7661
Row number: 1402 of 7661
Row number: 1403 of 7661
Row number: 1404 of 7661
Row number: 1405 of 7661
Row number: 1406 of 7661
Row number: 1407 of 7661
Row number: 1408 of 7661
Row number: 1409 of 7661
Row number: 1410 of 7661
Row number: 1411 of 7661


...

Row number: 6342 of 7661
Row number: 6343 of 7661
Row number: 6344 of 7661
Row number: 6345 of 7661
Row number: 6346 of 7661
Row number: 6347 of 7661
Row number: 6348 of 7661
Row number: 6349 of 7661
Row number: 6350 of 7661
Row number: 6351 of 7661
Row number: 6352 of 7661
Row number: 6353 of 7661
Row number: 6354 of 7661
Row number: 6355 of 7661
Row number: 6356 of 7661
Row number: 6357 of 7661
Row number: 6358 of 7661
Row number: 6359 of 7661
Row number: 6360 of 7661
Row number: 6361 of 7661
Row number: 6362 of 7661
Row number: 6363 of 7661
Row number: 6364 of 7661
Row number: 6365 of 7661
Row number: 6366 of 7661
Row number: 6367 of 7661
Row number: 6368 of 7661
Row number: 6369 of 7661
Row number: 6370 of 7661
Row number: 6371 of 7661
Row number: 6372 of 7661
Row number: 6373 of 7661
Row number: 6374 of 7661
Row number: 6375 of 7661
Row number: 6376 of 7661
Row number: 6377 of 7661
Row number: 6378 of 7661
Row number: 6379 of 7661
Row number: 6380 of 7661
Row number: 6381 of 7661


Row number: 6670 of 7661
Row number: 6671 of 7661
Row number: 6672 of 7661
Row number: 6673 of 7661
Row number: 6674 of 7661
Row number: 6675 of 7661
Row number: 6676 of 7661
Row number: 6677 of 7661
Row number: 6678 of 7661
Row number: 6679 of 7661
Row number: 6680 of 7661
Row number: 6681 of 7661
Row number: 6682 of 7661
Row number: 6683 of 7661
Row number: 6684 of 7661
Row number: 6685 of 7661
Row number: 6686 of 7661
Row number: 6687 of 7661
Row number: 6688 of 7661
Row number: 6689 of 7661
Row number: 6690 of 7661
Row number: 6691 of 7661
Row number: 6692 of 7661
Row number: 6693 of 7661
Row number: 6694 of 7661
Row number: 6695 of 7661
Row number: 6696 of 7661
Row number: 6697 of 7661
Row number: 6698 of 7661
Row number: 6699 of 7661
Row number: 6700 of 7661
Row number: 6701 of 7661
Row number: 6702 of 7661
Row number: 6703 of 7661
Row number: 6704 of 7661
Row number: 6705 of 7661
Row number: 6706 of 7661
Row number: 6707 of 7661
Row number: 6708 of 7661
Row number: 6709 of 7661


Row number: 7001 of 7661
Row number: 7002 of 7661
Row number: 7003 of 7661
Row number: 7004 of 7661
Row number: 7005 of 7661
Row number: 7006 of 7661
Row number: 7007 of 7661
Row number: 7008 of 7661
Row number: 7009 of 7661
Row number: 7010 of 7661
Row number: 7011 of 7661
Row number: 7012 of 7661
Row number: 7013 of 7661
Row number: 7014 of 7661
Row number: 7015 of 7661
Row number: 7016 of 7661
Row number: 7017 of 7661
Row number: 7018 of 7661
Row number: 7019 of 7661
Row number: 7020 of 7661
Row number: 7021 of 7661
Row number: 7022 of 7661
Row number: 7023 of 7661
Row number: 7024 of 7661
Row number: 7025 of 7661
Row number: 7026 of 7661
Row number: 7027 of 7661
Row number: 7028 of 7661
Row number: 7029 of 7661
Row number: 7030 of 7661
Row number: 7031 of 7661
Row number: 7032 of 7661
Row number: 7033 of 7661
Row number: 7034 of 7661
Row number: 7035 of 7661
Row number: 7036 of 7661
Row number: 7037 of 7661
Row number: 7038 of 7661
Row number: 7039 of 7661
Row number: 7040 of 7661


Row number: 7329 of 7661
Row number: 7330 of 7661
Row number: 7331 of 7661
Row number: 7332 of 7661
Row number: 7333 of 7661
Row number: 7334 of 7661
Row number: 7335 of 7661
Row number: 7336 of 7661
Row number: 7337 of 7661
Row number: 7338 of 7661
Row number: 7339 of 7661
Row number: 7340 of 7661
Row number: 7341 of 7661
Row number: 7342 of 7661
Row number: 7343 of 7661
Row number: 7344 of 7661
Row number: 7345 of 7661
Row number: 7346 of 7661
Row number: 7347 of 7661
Row number: 7348 of 7661
Row number: 7349 of 7661
Row number: 7350 of 7661
Row number: 7351 of 7661
Row number: 7352 of 7661
Row number: 7353 of 7661
Row number: 7354 of 7661
Row number: 7355 of 7661
Row number: 7356 of 7661
Row number: 7357 of 7661
Row number: 7358 of 7661
Row number: 7359 of 7661
Row number: 7360 of 7661
Row number: 7361 of 7661
Row number: 7362 of 7661
Row number: 7363 of 7661
Row number: 7364 of 7661
Row number: 7365 of 7661
Row number: 7366 of 7661
Row number: 7367 of 7661
Row number: 7368 of 7661


Row number: 7660 of 7661
Row number: 7661 of 7661


In [20]:
removal_matrix = (scores > THRESHOLD) & (scores < 100)
internal_matched = remove_repeat(unknown, removal_matrix)
internal_matched = internal_matched.reset_index(drop=True)

LOG: L AVIATION INSURANCE deleted!
LOG: AVIATION IN'SURANCE deleted!
LOG: AVLATLON INSURANCE deleted!
LOG: INSURANCE-ACCLDENT AND HEALTH deleted!
LOG: INSURANCE-ACCLDENT  HEALTH deleted!
LOG: LNSURANCE-ACCIDENT AND HEALTH deleted!
LOG: INSURANCE-ACCIDENT AND HE ALTH deleted!
LOG: VENETIAN BLINDS-MANUFAC- TURERS deleted!
LOG: ESTATES-OFFICES OF  deleted!
LOG: ESTATES-OFIICES OF  deleted!
LOG: ESTATES-OFTICES OF  deleted!
LOG: VENTLLATLNG SYSTEMS deleted!
LOG: VENTLLATING SYSTEMS deleted!
LOG: VENTILATING SYSTEM deleted!
LOG: FILTER MANUFAC TURERS deleted!
LOG: FASTENER MANUFACTURERS deleted!
LOG: CAICULATING MACHINES deleted!
LOG: MODEL MAKERS  MONEV TO LOA deleted!
LOG: MODEL MAKERS  M.ONEY TO LOA deleted!
LOG: GENERAL MERCHANDISE- WHOLESALE deleted!
LOG: GE NE RAL MERC HANDISE -WHOLE - SALE deleted!
LOG: GENERAL MERCHANDISE-W'HOLESALE deleted!
LOG: CONTRACTORY SUPPLIES AND EQUIPMENT deleted!
LOG: CONTRACTORS' SUPPLIES AND EQUIPMENT-RENTAL deleted!
LOG: CONTRACTORS SUPPLIES AND EQUIPME

LOG: SAFETY DEVICW AND EQUIPMENT deleted!
LOG: SHOE DEALERS-MENM deleted!
LOG: LE,QAL FORMS  STATIONERY deleted!
LOG: I-EQAL FORMS  STATIONERY deleted!
LOG: HEATING AND PLUMBING SUPPLIES deleted!
LOG: 'HEATING  PLUMBING SUPPLIES deleted!
LOG: HEATINN AND PLUMBING SUPPLIES deleted!
LOG: EY F HOAR CAMP L76 BENEFIT deleted!
LOG: LKPROVLSLON DEALERS  PUBLICATIONS deleted!
LOG: TILE CONTRACTORS  deleted!
LOG: IDR.AWING MATERIALS deleted!
LOG: RA-DIO ADVERTISING deleted!
LOG: CONTRAC TORS-ACOUSTICAL deleted!
LOG: CONTRACTORS-ACOUSTICAL deleted!
LOG: CONTRACTORS-ACOUSI'ICAL deleted!
LOG: CONTRACTORS -ACOUSTICAL deleted!
LOG: CONTRACTORS-ACOUBTICAL deleted!
LOG: BOOK PRINTERS deleted!
LOG: BOOVKLET PRINTERS deleted!
LOG: BFOOKLET PRINTERS deleted!
LOG: EMBLLEMS deleted!
LOG:  EM BLEMS deleted!
LOG: FORM MANUFACTURERS) deleted!
LOG: COMB MANUFACTURERS deleted!
LOG: PLYWOOD MANUFACTURERS deleted!
LOG: BROOM MANUFACTURERS deleted!
LOG: OOMICE MANUFACTURERS deleted!
LOG: DOOM MANUFACTURERS deleted

LOG: REPAIRERS) deleted!
LOG: PM REPAIRERS deleted!
LOG: BOAR DING  BOOMING HOUSES  BOILER MFRS deleted!
LOG: ANTHONY AV deleted!
LOG: BRALDLNG MACHINE MFRS deleted!
LOG: BRAIDLNG MACHINE MFRS deleted!
LOG: GRINDING MACHINERY MFRS deleted!
LOG: BRALDTNG MACHINE MFRS deleted!
LOG: RUSSETT LEATHER deleted!
LOG: LRUSSET LEATHER deleted!
LOG: REFRIGERATORS-BLECTRIC AND GAS deleted!
LOG: REFRIGERATORS-ELECTRIC  GAS deleted!
LOG: REFRIGERATORRS-BLECTRIC  GAS deleted!
LOG: REIRIGERATORS-ELECTRIC  GAS deleted!
LOG: F'REIRIGERATORS-ELECTRIC AND GAS deleted!
LOG: REFRIGERATORS-ELECTRIC AND GAS deleted!
LOG:  REFRIGERATORS -ELECTRIC AND GAS deleted!
LOG: RETRIGERATORS-ELEC,TRIC  GAS deleted!
LOG: HYDRAULIC EQUIPMENT AND SUPPLIES deleted!
LOG: HYDRAULIC EQUIPMENT BND SUPPLIES deleted!
LOG: HYDRAULIC EQUIPMEN T AND SUPPLIES deleted!
LOG: ELECTRICIANS  ELECTROLYTIC TREATMENT deleted!
LOG: FASTENERS-MTRS deleted!
LOG: FASTENERS-MIRS deleted!
LOG: PASTENERS-MIRS deleted!
LOG: RUBBER GGODS- MANUFACTURE

LOG: CREAMERIES  GREDIT JEWELERS deleted!
LOG: SPECIAL MACHINERY deleted!
LOG: GARAGE DOORS- DEALERS AND DISTRIBUTORS deleted!
LOG: HAIR GOODS-EUMAN-MIRS  DEALERS deleted!
LOG: HAIR GOODS-HUMAN-JMRS  DEALERS deleted!
LOG: HAIR GOODS-HUMAN-MTRS  DEALERS deleted!
LOG: VENETIANBLINDMFRS deleted!
LOG: VENETIAN BLLND-MFRS deleted!
LOG: BLESCTRIC IRONERS deleted!
LOG: ELECTRIC IRONETRS deleted!
LOG:  EXTE RMINATIPG deleted!
LOG: EXTERMLNATING deleted!
LOG: EXTERMINATLNG deleted!
LOG: A EXTERMINATING deleted!
LOG: FRUIT BUYERS PACKERS AND SHIPPERS deleted!
LOG: OLL BURNING EQUIPMENT deleted!
LOG: F'OIL BURNING EQUIPMENT deleted!
LOG: LIIE INSURANCE ANNUITIES deleted!
LOG: LIFE INSURANCE ANNUITIES deleted!
LOG: BAN MIRS-CANVAS, BURLAP  COTTO deleted!
LOG: BAG MIRS-FGANVAS, BURLAP  COTTO deleted!
LOG: CERTIIIED PUBLIC ACCOUNTANTS  CHAIN MFRS deleted!
LOG: CERTIIIED PUBLIC ACCOUNTANTS  C'HAIN MFRS deleted!
LOG: CERTIFIED PUBLIC ACCOUNTANTS  CHAIN MFRS deleted!
LOG: VOERTLFIED PUBLIC ACCOUNTANTS 

LOG: EATING HAUSA  EGG DEALERS-WHOLESALE deleted!
LOG: WATER W ORKS SYSTEMS deleted!
LOG: EATING HAUSA  EGG DEALERS-WHOLESALE deleted!
LOG: 'GRAVE MARKERS deleted!
LOG: TORS -MANUFAC TURERS deleted!
LOG: APOTHECARIES  APPRAISERS deleted!
LOG: GASOLINE STATION EQUIPME NT deleted!
LOG: GAS  FILLING STATION EQUIPMENT deleted!
LOG: IHOLDING COMPANIES deleted!
LOG: GONCRETE--READV MIXED deleted!
LOG: COMLCRETE-READY MIXED deleted!
LOG: CONCRETE -READY MIXED deleted!
LOG: AUTOMOBILE BODY MANUFACT URE RS deleted!
LOG: ARTIFICIAL FLOWERS AND FRUIT MANUFACTURE RS deleted!
LOG: ARTIFIC IAL FLOWERS AND FRUIT MANUFACTURERS deleted!
LOG: ARTIFICIAL FLOWERS AND' FRUIT MANUFACTURERS deleted!
LOG: TRUNKS  TRAVELING BAGS RETAIL deleted!
LOG: TRUNKS  TRAVELING BAUS- RETAIL deleted!
LOG: TMNKS  TRAVELING BAGS- RETAIL deleted!
LOG: TEACHERS  TEACHERS-ELOCRUTLO deleted!
LOG: COLN DEALERS  ' COKE-RETALL deleted!
LOG: HARDWARE MANUFAC TURERS deleted!
LOG: ILP'LUMBING FIXTURES deleted!
LOG: '-'PLUMBING FIXTUR

...

LOG: PRINTERS-GRAVURE deleted!
LOG: JEWELERS2 ENAMELS deleted!
LOG: JEWELEVRS2 ENAMELS deleted!
LOG: ACIDS-I'ATTY deleted!
LOG: AUTOMOBLLE ASSOC IATIONS  deleted!
LOG: BUIL.DERS' FINISH deleted!
LOG: CIOMMERCIAL SIGNS deleted!
LOG: ELECTRIC LIGHT AND POWER. COMPANIES deleted!
LOG: ELECTRIC LIGHT AND POWER COM PANIES deleted!
LOG: LOANS-PURNITURE  HOUSE- HOLD GOODS deleted!
LOG: LOANS-PURNITURE  HOUSE- HOLD GOODS  deleted!
LOG: 'PRINTERS-LABEL deleted!
LOG: DRWS SUITS RENTED deleted!
LOG: PARTITIONS-STEEL deleted!
LOG:  ARTISTS' PAINTS deleted!
LOG: LK'PRINTERS-THE SAILOR'S GUIDE deleted!
LOG: PRINTERS-THE SAILOR'S GUIDE deleted!
LOG: BROOMS, BRUSHES  MOPS-MFRS deleted!
LOG: BROOMS, BRUSHES  MAPS-MIRS deleted!
LOG: BROOMS, BRUSHES  MOPS-MTRS deleted!
LOG: CONVEYING SYSTE MS deleted!
LOG: CONVEYING SYSTEMS deleted!
LOG: AIR CONDITIONERS -ROOM deleted!
LOG: ICE DEALERS-PE'DDLERS SUPPLIED deleted!
LOG: MONEV TRANSPORTATIO deleted!
LOG: PACKING-STEAM, ETC deleted!
LOG: CONTRACTORS' EQUIPMEN

LOG: PENCES-WIRE-IRON deleted!
LOG: I'ENCESR-WIRE-IRON deleted!
LOG: TOILE T PRE PARATIONS - MANUFACTURERS deleted!
LOG: I'LOOR SURFACING  COVERING deleted!
LOG: LINENS-RETAIL  LINOTYPE  MONOTYPE GOM- POSITIMI deleted!
LOG: HEATINN AND PLUMBING SUPPLIES deleted!
LOG: CHINA, CROCKERY, GLASSWARE AND EAR THENWARE-WHOLESALEE- SALE deleted!
LOG: CHINA, CROCKERY, GLABSWARE AND EARTHENWARE deleted!
LOG: CHINA, CROCKERY, GLASSWARE AND EARTHE NWARE-WHOLE- SALE deleted!
LOG: AUTOINOBILE FILLING STATIONS  AUTO FRAME STRAIGHTENING deleted!
LOG: BURGLAR  FIRE ALARM NOTIFICA- TION, WATCHMAN'S SUPERVISON SERVICE deleted!
LOG: BURGLAR AND FIRE ALARM NOTIFICATION, WATCHMAN2S SUPERVISORY SERVICE deleted!
LOG: BURGLAR  FIRE ALARM NOTIFICA- TION, WATCHMAMS SUPERVISORY SERVICE deleted!
LOG: BURGLAR AND FIRE ALARM NOTIFICATION, WATCHMAN,S SUPERVISORY SERVICE deleted!
LOG: STAMPS AND STENCILS-RUBBER deleted!
LOG:  FOREIGN BANKING deleted!
LOG: POREIGN BANKING deleted!
LOG: FOREIGN BANKING deleted!
LOG: PACKA

LOG: MEN'S FURNISHING GOODS deleted!
LOG: HALLS-BANQUET deleted!
LOG: VCARPET CLEANERS . LAYERS  GARPET LINING deleted!
LOG: CONTRACTORS-ACOUSI'ICAL deleted!
LOG: CONTRACTORS -ACOUSTICAL deleted!
LOG: CONTRACTORS-ACOUBTICAL deleted!
LOG:  MOVING- LOCAL AND LONG DISTANCE deleted!
LOG:  MOVING-LOCAL AND LON G DISTANCE deleted!
LOG: CONCRETERS AND ROOFERS deleted!
LOG: ROOFING IRON - GALVANIZED  CORRUGATED deleted!
LOG: NLCKEL PLATERS  NICKEL SILVER MFRS deleted!
LOG: KNIT GOODS- MANUF CTURERS deleted!
LOG: RADIW AND TELEVISION deleted!
LOG: ELECTRIC MOTORS AND  GENERATORS-MFRS deleted!
LOG: GREETLNG GARDS-MFRS deleted!
LOG: CONTRACTORS-ASPHALT PAVING deleted!
LOG: LCONTRACTORSMASPHAL T PA VING deleted!
LOG: LADIES,  MISSES READY TO WEAR GLOTHING-RETAIL  deleted!
LOG: LADIES4 AND MISSEQ READY-TO- WEAR CLOTHING-RETAIL deleted!
LOG: MUSICIANS  N AIL MFRS deleted!
LOG: JEWELEVRS2 ENAMELS deleted!
LOG: SODA  MINERAL WATER MFRS  SOLDER  SOLDERING PLUXES-MFRS deleted!
LOG: SODA  MINERAL WATER M

LOG:  CLOTHING-CREDIT deleted!
LOG: PURNACES-IMDUSTRIAL MFRS deleted!
LOG: FFGALVANIZED SHEET IRON ASH CANS AND PAILS deleted!
LOG: '- GALVANIZED SHEET IRON ASH CANS AND PAILS deleted!
LOG: GALVANIZED SHEET IRON ASH   ' CANS  PAILS deleted!
LOG: IWGALVANIZED SHEET IRON ASH CANS AND PAILS deleted!
LOG: PUHLLC UTILITIES deleted!
LOG:  INSURANCE-JEWELRY deleted!
LOG: AUTOMOBILE PAINTERS  AUTOMOBILE PAINTERS deleted!
LOG: AUTOMOBILE LIVERIES  AUTOMOBLLE PAINTERS deleted!
LOG: LOANS-SINQLE SIGNATURE deleted!
LOG: FFCONTRACTLORS-HEAVY MACHINERY MOVERS deleted!
LOG: ASPHALT  CONCRETE PAVEMENTS deleted!
LOG: REAL ESTATE-SALES  RENT'ALS deleted!
LOG: LIME, BRICK, CEMENT, SAND, HAIR, PLASTER ETC  deleted!
LOG: LIME, BRICK. CEMENT SAND, HAIR, PLASTER C deleted!
LOG: ELEACHERS-FABRICS deleted!
LOG: POLDLNG BOX MANUFACTURERS deleted!
LOG: MOLDING MANUFACTURERS deleted!
LOG: SHLPPING TAG MFRS deleted!
LOG: POREIGN BANKING deleted!
LOG: FOREIGN BANKING deleted!
LOG: MACHINISTS' SUPPLIES AND TOOL TOOL

LOG: GRANLTE DEALERS deleted!
LOG: REAL ESTATE -COMMERC - IAL deleted!
LOG: OTTICE  TOILET PARTITIONS- METAL deleted!
LOG: OTTICE  TOILET PARTITIONS- METAL  WOOD deleted!
LOG: THEATRIEAL AGENTS  THREAD MIRS-GOTTON ETC deleted!
LOG: FEDERAL SAVINGS AND LOAN ASSOCIATKNS deleted!
LOG: OIL BURNER DEALERSL SUPPLIES deleted!
LOG: LOANS-PURNITURE  HOUSE- HOLD GOODS  deleted!
LOG: MEN'S PUMISHLNGS-I'ULL DRESS deleted!
LOG: MEN'S PURNLSHLNGS-L'ULL DRESS deleted!
LOG: LADIES4 AND MISSEQ READY-TO- WEAR CLOTHING-RETAIL deleted!
LOG: PIP. AND SMOKERS' ARTICLES-DEALERS deleted!
LOG: UTLLITIES-PUBLIC deleted!
LOG: LINENS-RETAIL  LINOTYPE  MONOTYPE GOM- POSITIMI deleted!
LOG: FUMIGATING deleted!
LOG: WAREHOUSW-HOUSEIHOLD GOODS deleted!
LOG: GLASS DEALERS-RETAIL deleted!
LOG: MONSTRUCTIOFN LOANS deleted!
LOG: LOANS--I'ARM deleted!
LOG:  'INSURANCE-WAR  BOMBARD- MENT deleted!
LOG: EMBLEMS-RELIGIOUS  FRA- TE-RNAL deleted!
LOG: GARAUES-STEEL deleted!
LOG: V'ENETIAN BLIND CLEANERS AND RE PAIRE RS deleted!
